---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    state_name = None
    states_town_dict = {'State': [], 'RegionName': []}
    for name in open('university_towns.txt', 'r'):
        name = name.rstrip()
        if name.endswith('[edit]'):
            a = name
            continue
        states_town_dict['State'].append(a)
        states_town_dict['RegionName'].append(name)
    states_town = pd.DataFrame(states_town_dict, columns=['State', 'RegionName'])
    states_town['State'].replace(r'^([^\[\]\d]+).*$', r'\1', regex=True, inplace=True)
    states_town['RegionName'].replace(r'^([^\(\)\d]+).*$', r'\1', regex=True, inplace=True)
    states_town['State'] = states_town['State'].str.strip()
    states_town['RegionName'] = states_town['RegionName'].str.strip()
    return states_town
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows=219, usecols=(4, 5, 6), names=['Quarter', 'GDP Current', 'GDP 2009'])
    gdp_current_list = gdp['GDP Current'].tolist()
    recession_index = None
    for i in range(2, len(gdp_current_list)):
        if gdp_current_list[i - 2] > gdp_current_list[i - 1] > gdp_current_list[i]:
            recession_index = i-2
            break
    return gdp.iloc[recession_index]['Quarter']
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=219, usecols=(4, 5, 6), names=['Quarter', 'GDP Current', 'GDP 2009'])
    gdp_current_list = gdp['GDP Current'].tolist()
    recession_start_index = gdp[gdp['Quarter'] == get_recession_start()].index[0]
    recession_end_index = None
    for i in range(34, len(gdp_current_list)):
        if gdp_current_list[i - 2] > gdp_current_list[i - 1] > gdp_current_list[i]:
            recession_end_index = i
    #print(gdp.iloc[33:(recession_end_index+3)])
    return gdp.iloc[recession_end_index+2]['Quarter']
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows=219, usecols=(4, 5, 6), names=['Quarter', 'GDP Current', 'GDP 2009'])
    gdp_current_list = gdp['GDP Current'].tolist()
    recession_start_index = gdp[gdp['Quarter'] == get_recession_start()].index[0]
    recession_end_index = gdp[gdp['Quarter'] == get_recession_end()].index[0]
    recession_bottom_value = gdp['GDP Current'].iloc[recession_start_index:recession_end_index+1].min()
    return gdp['Quarter'][gdp['GDP Current'] == recession_bottom_value].values[0]
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    l = housing_data.columns.tolist()
    housing_dates = l[51:]

    quarters = housing_data[housing_dates].groupby(pd.PeriodIndex(housing_data.columns[51:], freq='Q'), axis=1).mean()

    housing_data.drop(l[6:], axis=1, inplace=True)
    housing_data.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace=True)
    housing_data[quarters.columns.tolist()] = quarters
    states_list = housing_data['State'].tolist()

    for i, value in enumerate(states_list):
        states_list[i] = states[value]
    housing_data['State'] = states_list
    
    housing_data.set_index(['State', 'RegionName'], inplace=True)
    housing_data.columns = housing_data.columns.values.astype('str')
    housing_data.columns = map(str.lower, housing_data.columns)
    return housing_data
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #load data
    university_towns_list = get_list_of_university_towns()
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_bottom = get_recession_bottom()
    housing_data = convert_housing_data_to_quarters()
    gdp = pd.read_excel('gdplev.xls', skiprows=219, usecols=(4, 5, 6), names=['Quarter', 'GDP Current', 'GDP 2009'])
    
    #create new data showing the decline or growth of housing prices between the recession start and the recession bottom
    recession_period_list = []
    housing_data_columns_list = housing_data.columns.tolist()
    index_recession_start = housing_data_columns_list.index(recession_start)
    index_recession_bottom = housing_data_columns_list.index(recession_bottom)
    index_recession_end = housing_data_columns_list.index(recession_end)
    #print(f'index_recession_start: {index_recession_start}\nindex_recession_bottom: {index_recession_bottom}\nindex_recession_end: {index_recession_end}\n')
    

    housing_data['ratio'] = housing_data['2008q2'] / housing_data['2009q2']
    university_towns_list.set_index(['State', 'RegionName'], inplace=True)
    
    
    #getting recession data for university towns and non-university towns
    non_university_towns_houses = housing_data[~housing_data.index.isin(university_towns_list.index)]
    university_towns_houses = pd.merge(university_towns_list, housing_data, how='inner', left_index=True, right_index=True)
    #print(f'non_university_towns_houses: {len(non_university_towns_houses)},\nuniversity_towns_houses: {len(university_towns_houses)}\n')
    
    
    
    #mean
    non_university_towns_houses_mean = non_university_towns_houses['ratio'].mean()
    university_towns_houses_mean = university_towns_houses['ratio'].mean()
    #print(f'non_university_towns_houses_mean: {non_university_towns_houses_mean}\nuniversity_towns_houses_mean: {university_towns_houses_mean}\n')
    
    #running ttest
    ttest_results = ttest_ind(non_university_towns_houses['ratio'], university_towns_houses['ratio'], nan_policy='omit')
    result = None

    if ttest_results.pvalue < 0.01:
        result = (True, ttest_results.pvalue, 'non-university town' if non_university_towns_houses_mean < university_towns_houses_mean else 'university town')
    else:
        result = (False, ttest_results.pvalue, 'non-university town' if non_university_towns_houses_mean < university_towns_houses_mean else 'university town')
    return result
run_ttest()

(True, 0.0027240637047531249, 'university town')